In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install surprise

     |████████████████████████████████| 11.8 MB 14.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619411 sha256=9ae4e7876b14d561cf73d5b7c7c7882384ffc39cd239a294014c2a456dd8b33b
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
import os
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error


import xgboost as xgb
from surprise import Reader, Dataset
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SVD
from surprise import accuracy
from surprise import SVDpp
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV


In [4]:
df = pd.read_csv("/content/drive/MyDrive/NetflixProject/Netflix/archive/NetflixRatings.csv", sep=",", names = ["MovieID","CustID", "Ratings", "Date"])
df["Date"] = pd.to_datetime(df["Date"])
df_movie=pd.read_csv('/content/drive/MyDrive/Netflix/Data/additions/updated_movie_titles.csv')

ID_values=df_movie[df_movie['Year'].isnull()].ID.values
years=[]
for i in ID_values:
  temp=df[df['MovieID']==i].sort_values('Date').iloc[0]
  years.append(temp.Date.year)
temp1=df_movie[df_movie['Year'].isnull()]
count=0
for i in years:
  temp1.Year.values[count]=i
  count=count+1
df_movie.dropna(inplace=True)
df_movie=pd.concat([df_movie,temp1])
df_movie.sort_index(inplace=True)
df_movie.rename(columns = {'ID':'MovieID'}, inplace = True)

In [5]:
dfnew=df.copy()

In [6]:
dfnew=dfnew.merge(df_movie,on='MovieID')
dfnew.head()

,MovieID,CustID,Ratings,Date,Year,Name
0,1,1488844,3,2005-09-06,2003.0,Dinosaur Planet
1,1,822109,5,2005-05-13,2003.0,Dinosaur Planet
2,1,885013,4,2005-10-19,2003.0,Dinosaur Planet
3,1,30878,4,2005-12-26,2003.0,Dinosaur Planet
4,1,823519,3,2004-05-03,2003.0,Dinosaur Planet


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
#train=dfnew.sample(50000000)
train=dfnew

In [9]:
train.shape

(100480507, 6)

In [10]:
train.head()

,MovieID,CustID,Ratings,Date,Year,Name
0,1,1488844,3,2005-09-06,2003.0,Dinosaur Planet
1,1,822109,5,2005-05-13,2003.0,Dinosaur Planet
2,1,885013,4,2005-10-19,2003.0,Dinosaur Planet
3,1,30878,4,2005-12-26,2003.0,Dinosaur Planet
4,1,823519,3,2004-05-03,2003.0,Dinosaur Planet


Sparse matrix


In [11]:
# Creating a train sparse matrix
train_sparse_matrix = sparse.csr_matrix((train.Ratings.values, (train.CustID.values, train.MovieID.values)))

In [12]:
rows,cols = train_sparse_matrix.shape
presentElements = train_sparse_matrix.count_nonzero()

print("Sparsity Of Train matrix : {}% ".format((1-(presentElements/(rows*cols)))*100))

Sparsity Of Train matrix : 99.78658865580644% 


In [13]:
train_sparse_matrix.shape

(2649430, 17771)

Movie-Movie Similarity Matrix

In [14]:
df_movie.set_index('MovieID',drop=True,inplace=True)

In [15]:
movie_similar= cosine_similarity(train_sparse_matrix.T, dense_output = False)

In [16]:
movie_ids = np.unique(movie_similar.nonzero())

similar_movies_dict = dict()
for movie in movie_ids:
    smlr = np.argsort(-movie_similar[movie].toarray().ravel())[1:100]
    similar_movies_dict[movie] = smlr

In [17]:
movieID_GR = 17765

print("Name of the movie -------> "+str(df_movie.loc[movieID_GR][1]))

print("Number of ratings by users for movie {} is {}".format(df_movie.loc[movieID_GR][1], train_sparse_matrix[:,movieID_GR].getnnz()))

print("Number of similar movies to {} is {}".format(df_movie.loc[movieID_GR][1], movie_similar[movieID_GR].count_nonzero()))

Name of the movie -------> Godzilla's Revenge
Number of ratings by users for movie Godzilla's Revenge is 313
Number of similar movies to Godzilla's Revenge is 17749


In [18]:
df_movie.loc[similar_movies_dict[movieID_GR]][:20]

,Year,Name
MovieID,,
15810,1964.0,Godzilla vs. Mothra
12252,1975.0,Terror of Mechagodzilla
5907,1956.0,Godzilla: King of the Monsters
2459,1967.0,Son of Godzilla
409,1966.0,Godzilla vs. The Sea Monster
14623,1971.0,Godzilla vs. Hedorah
2442,1972.0,Godzilla vs. Gigan
17746,1991.0,Godzilla & Mothra: Battle for Earth / Vs. King...
8233,1968.0,Destroy All Monsters
